In [1]:
import csv
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as plcolor

In [2]:
def my_layout(G, scale=100):
    layout = {}
    for node in G.nodes():
        layout[node] = [G.nodes[node]['x'] / scale, G.nodes[node]['y'] / scale]
    return layout

In [3]:
cdict = {
    'red'   : [[0.0, 0.0, 0.0],
               [1.0, 0.0, 0.0]],
    'green' : [[0.0, 0.0, 0.0],
               [1.0, 0.0, 0.0]],
    'blue'  : [[0.0, 0.0, 0.0],
               [1.0, 0.0, 0.0]],
    'alpha' : [[0.0, 0.0, 0.0],
               [1.0, 1.0, 1.0]]
}
black_alpha = plcolor.LinearSegmentedColormap('black_alpha', cdict)

In [13]:
def individual_describe(G, weight_str, dist_str):
    cluster_dict = nx.clustering(G, weight=weight_str)
    central_dict = nx.closeness_centrality(G, distance=dist_str, wf_improved=True)
    ind_list = []
    for n in G.nodes():
        if not 'x' in G.nodes[n].keys():
            G.nodes[n]['x'] = None
        if not 'y' in G.nodes[n].keys():
            G.nodes[n]['y'] = None
        dict_des = {'clustering': cluster_dict[n], 'out_degree': G.out_degree(n), 'avg_position_x': G.nodes[n]['x'], 'avg_position_y': G.nodes[n]['y'], 'centrality': central_dict[n], 'PlayerID': n}
        ind_list.append(dict_des)
    return ind_list

In [14]:
def gen_cal_graph_match(match_num, team_name):
    G = nx.DiGraph()
    with open('2020_Problem_D_DATA/weight.csv') as f:
        f_csv = csv.DictReader(f)
        for row in f_csv:
            if int(row['MatchID']) is int(match_num) and team_name in row['OriginID'] and team_name in row['DestinationID']:
                if G.has_edge(row['OriginID'], row['DestinationID']):
                    G[row['OriginID']][row['DestinationID']]['weight'] += int(row['Weight'])
                else:
                    G.add_edge(row['OriginID'], row['DestinationID'], weight=int(row['Weight']))

    with open('2020_Problem_D_DATA/location.csv') as f:
        f_csv = csv.DictReader(f)
        for row in f_csv:
            if row['OriginPlayerID'] in G.nodes() and int(row['MatchID']) is int(match_num):
                G.nodes[row['OriginPlayerID']]['x'] = float(row['EventOrigin_x'])
                G.nodes[row['OriginPlayerID']]['y'] = float(row['EventOrigin_y'])
    for u, v in G.edges():
        G[u][v]['distance'] = 1 / G[u][v]['weight']

    total_list = individual_describe(G, 'weight', 'distance')
    for dict_ele in total_list:
        dict_ele['teamID'] = team_name
        dict_ele['matchID'] = match_num
    return total_list

In [15]:
total_list = []
with open('2020_Problem_D_DATA/matches.csv') as f:
    f_csv = csv.DictReader(f)
    for row in f_csv:
        total_list.extend(gen_cal_graph_match(row['MatchID'], 'Huskies'))
        # total_list.extend(gen_cal_graph_match(row['MatchID'], row['OpponentID']))

In [17]:
headers = ['matchID', 'teamID', 'PlayerID', 'clustering', 'out_degree', 'centrality', 'avg_position_x', 'avg_position_y']
with open('match_indicator_individual.csv', 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(total_list)